In [53]:
# Import required packages 
import os
import glob
import cv2
import numpy as np
import pandas as pd
import random

# Read the training data file with error handling for bad lines
# data = pd.read_excel("D:\\updated_final_book_1.xlsx")  #path on windows
data = pd.read_excel("/Users/almazbaghirova/Desktop/Hiwi/VideoCut/test_videos.xlsx")


# Display the first few rows
print(data.head())
print(list(data.columns))
data.info()


def convert_path(path):
    # Remove 'file://' prefix if present
    if path.startswith('file://'):
        path = path[7:]
    # Normalize the path and ensure it's in the correct format
    path = os.path.normpath(path)
    return path

# Check dimensions of the subset 
print(data.shape)
data['HighRes_Drive_Path'] = data['HighRes_Drive_Path'].apply(lambda x: convert_path(x.strip()))


        Date  Obs_no Flight  Total_flights  Start_time  End_time    Drone  \
0 2023-08-07       1      A              1         NaN       NaN  Pink_M3   
1 2023-08-07       1      A              1         NaN       NaN  Pink_M3   
2 2023-08-07       1      A              1         NaN       NaN  Pink_M3   
3 2023-08-07       1      A              1         NaN       NaN  Pink_M3   

  Pilot Location     Video  ... group size start   end  light quality   \
0    AD     GCP1  DJI_0404  ...        NaN  10.0  50.0             NaN   
1    AD     GCP1  DJI_0405  ...        NaN   NaN   NaN             NaN   
2    AD     GCP1  DJI_0406  ...        NaN   NaN   NaN             NaN   
3    AD     GCP1  DJI_0407  ...        NaN   NaN   NaN             NaN   

   habitat                      unique_id  \
0      NaN  0708_001_PM3_GCP1_AD_DJI_0404   
1      NaN  0708_001_PM3_GCP1_AD_DJI_0405   
2      NaN  0708_001_PM3_GCP1_AD_DJI_0406   
3      NaN  0708_001_PM3_GCP1_AD_DJI_0407   

                 

In [54]:
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
import os

def convert_path(path):
    # Remove 'file://' prefix if present
    if path.startswith('file://'):
        path = path[7:]
    # Normalize the path and ensure it's in the correct format
    path = os.path.normpath(path)
    return path

# Define a function to trim the video
def trim_video(row):
    video_path = convert_path(row['HighRes_Drive_Path'].strip())  # Normalize and strip any leading/trailing whitespace
    if video_path.startswith("\\"):
        video_path = video_path[1:]  # Remove leading backslash if present
    start_time = row['start']
    end_time = row['end']
    unique_id = row['unique_id']
    
    try:
        # Load the video
        with VideoFileClip(video_path) as video:
            if start_time == "start" and pd.notnull(end_time) and end_time != "end":
                end_time = float(end_time)
                trimmed_video = video.subclip(0, end_time)
            elif end_time == "end" and pd.notnull(start_time) and start_time != "start":
                start_time = float(start_time)
                trimmed_video = video.subclip(start_time, video.duration)
            elif start_time == "start" and end_time == "end":
                trimmed_video = video
            elif pd.notnull(start_time) and pd.notnull(end_time) and start_time != "start" and end_time != "end":
                start_time = float(start_time)
                end_time = float(end_time)
                trimmed_video = video.subclip(start_time, end_time)
            else:
                raise ValueError(f"Invalid start or end time for unique ID: {unique_id}")

            # Define the output directory and filename
            output_folder = os.path.join(output_directory, unique_id)
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{unique_id}_P01.mp4"
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the trimmed video with the same quality as the original
            trimmed_video.write_videofile(output_path, codec='libx264', preset='ultrafast', audio_codec='aac', temp_audiofile='temp-audio.m4a', remove_temp=True)
            print(f"Successfully processed: {output_filename}")

    except Exception as e:
        print(f"Error processing {unique_id}: {e}")

# Function to process videos by unique IDs
def process_videos_by_ids(unique_ids):
    for unique_id in unique_ids:
        unique_id = unique_id.strip()  # Remove any leading/trailing whitespace
        # Debug statement to check the unique ID
        print(f"Processing unique ID: {unique_id}")

        # Find the row corresponding to the unique ID
        selected_row = data[data['unique_id'].astype(str).str.strip() == unique_id]
        
        if not selected_row.empty:
            video_path = selected_row['HighRes_Drive_Path'].values[0]
            print(f"Found video path: {video_path}")
            selected_row.apply(trim_video, axis=1)
        else:
            print(f"No video found with the unique ID: {unique_id}")

import random
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def extract_frames(video_path, output_folder):
    video = VideoFileClip(video_path)
    duration = int(video.duration)

   
    frames_to_extract = int(duration / 5)  # New: Calculate the number of frames to extract as 1/3 of the total duration
    frame_indices = sorted(random.sample(range(duration), frames_to_extract))  # New: Select 1/3 random frame indices

    # for sec in range(duration):
    #     frame_time = sec + random.random()  # Random time within each second
    #     frame = video.get_frame(frame_time)
    #     frame_filename = f"{os.path.basename(video_path).split('.')[0]}_{sec+1}_{int(frame_time*1000)}.png"
    #     frame_path = os.path.join(output_folder, frame_filename)
    #     video.save_frame(frame_path, t=frame_time)
    #     print(f"Extracted frame at {frame_time:.2f}s to {frame_path}")

    # New: Loop through only the selected frame indices
    for sec in frame_indices:
        frame_time = sec + random.random()  # Random time within each second
        frame = video.get_frame(frame_time)
        frame_filename = f"{os.path.basename(video_path).split('.')[0]}_{sec+1}_{int(frame_time*1000)}.png"
        frame_path = os.path.join(output_folder, frame_filename)
        video.save_frame(frame_path, t=frame_time)
        print(f"Extracted frame at {frame_time:.2f}s to {frame_path}")

# Example usage
def process_extracted_videos(unique_ids):
    for unique_id in unique_ids:
        unique_id = unique_id.strip()  # Remove any leading/trailing whitespace
        # Define the output folder
        output_folder = os.path.join(output_directory, unique_id)
        video_filename = f"{unique_id}_P01.mp4"
        video_path = os.path.join(output_folder, video_filename)
        
        if os.path.exists(video_path):
            print(f"Processing extracted frames for video: {video_path}")
            extract_frames(video_path, output_folder)
        else:
            print(f"No trimmed video found for unique ID: {unique_id}")



In [55]:

# Ensure the output directory exists
# output_directory = "D:\\wolf_drone_data\\cut_videos" #path on windows
output_directory = "/Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos"

# Define your list of unique IDs (copy multiple IDs) 
unique_ids = ['0708_001_PM3_GCP1_AD_DJI_0404']


In [56]:
# # Process/ trim the videos by unique ID
process_videos_by_ids(unique_ids)

Processing unique ID: 0708_001_PM3_GCP1_AD_DJI_0404
Found video path: /Users/almazbaghirova/Desktop/Hiwi/VideoCut/videos/DJI_0404.MP4


Moviepy - Building video /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4.
Moviepy - Writing video /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4



Moviepy - Done !
Moviepy - video ready /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4
Successfully processed: 0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4


In [57]:
# Process the extracted frames for the trimmed videos
process_extracted_videos(unique_ids)

Processing extracted frames for video: /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4
Extracted frame at 0.85s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_1_853.png
Extracted frame at 8.93s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_9_8927.png
Extracted frame at 18.08s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_19_18075.png
Extracted frame at 22.45s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_23_22446.png
Extracted frame at 23.37s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_24_23366.png
Extracted frame at 25.28s to /Users/alma